In [36]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv('final.csv')

In [38]:
df = df.dropna()
df.shape

(1870, 4)

In [39]:
df['Spread'] = df['banknifty']-df['nifty']
df['z_score'] = (df['Spread']-df['Spread'].rolling(window=50).mean())/df['Spread'].rolling(window =50).std()
df['PnL'] = df['Spread'] * (df['tte'] ** 0.7)
print(df['z_score'].max())
print(df['z_score'].min())
df

4.375922683415038
-3.892457459496086


,time,banknifty,nifty,tte,Spread,z_score,PnL
0,2021-01-04 09:15:00,0.265961,0.185760,24,0.080201,NaN,0.741870
1,2021-01-04 09:16:00,0.261887,0.184652,24,0.077235,NaN,0.714434
2,2021-01-04 09:17:00,0.260947,0.184509,24,0.076439,NaN,0.707066
3,2021-01-04 09:18:00,0.261434,0.183968,24,0.077466,NaN,0.716571
4,2021-01-04 09:19:00,0.259671,0.183756,24,0.075915,NaN,0.702219
...,...,...,...,...,...,...,...
1875,2021-01-08 15:26:00,0.249490,0.170833,20,0.078657,-0.289426,0.640410
1876,2021-01-08 15:27:00,0.250418,0.171149,20,0.079269,-0.009615,0.645397
1877,2021-01-08 15:28:00,0.251200,0.172256,20,0.078945,-0.290692,0.642755
1878,2021-01-08 15:29:00,0.250836,0.172420,20,0.078417,-0.663280,0.638452


In [44]:
entry_threshold = 2
exit_threshold = -2

df['Long_Entry'] = np.where(df['z_score'] < -entry_threshold, 1, 0)
df['Short_Entry'] = np.where(df['z_score'] > entry_threshold, 1, 0)
df['Long_Exit'] = np.where(df['z_score'] > -exit_threshold, 1, 0)
df['Short_Exit'] = np.where(df['z_score'] < exit_threshold, 1, 0)
df

,time,banknifty,nifty,tte,Spread,z_score,PnL,Long_Entry,Short_Entry,Long_Exit,Short_Exit
0,2021-01-04 09:15:00,0.265961,0.185760,24,0.080201,NaN,0.741870,0,0,0,0
1,2021-01-04 09:16:00,0.261887,0.184652,24,0.077235,NaN,0.714434,0,0,0,0
2,2021-01-04 09:17:00,0.260947,0.184509,24,0.076439,NaN,0.707066,0,0,0,0
3,2021-01-04 09:18:00,0.261434,0.183968,24,0.077466,NaN,0.716571,0,0,0,0
4,2021-01-04 09:19:00,0.259671,0.183756,24,0.075915,NaN,0.702219,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1875,2021-01-08 15:26:00,0.249490,0.170833,20,0.078657,-0.289426,0.640410,0,0,0,0
1876,2021-01-08 15:27:00,0.250418,0.171149,20,0.079269,-0.009615,0.645397,0,0,0,0
1877,2021-01-08 15:28:00,0.251200,0.172256,20,0.078945,-0.290692,0.642755,0,0,0,0
1878,2021-01-08 15:29:00,0.250836,0.172420,20,0.078417,-0.663280,0.638452,0,0,0,0


In [45]:
df['Position_Size'] = 0
long_position = False
short_position = False
for i in range(len(df)-1):
    if df['Long_Entry'].iloc[i] == 1 and not long_position:
        df['Position_Size'].iloc[i] = 1
        long_position = True
    elif df['Short_Entry'].iloc[i] == 1 and not short_position:
        df['Position_Size'].iloc[i] = -1
        short_position = True
    elif df['Long_Exit'].iloc[i] == 1 and long_position:
        df['Position_Size'].iloc[i] = 0
        long_position = False
    elif df['Short_Exit'].iloc[i] == 1 and short_position:
        df['Position_Size'].iloc[i] = 0
        short_position = False

<ipython-input-45-89670471de96>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Position_Size'].iloc[i] = 1
<ipython-input-45-89670471de96>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Position_Size'].iloc[i] = -1
<ipython-input-45-89670471de96>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Position_Size'].iloc[i] = 0
<ipython-input-45-89670471de96>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [46]:
df=df.dropna()

In [53]:
df['Strategy_Returns'] = df['PnL'].pct_change() * df['Position_Size'].shift(1)
df['cummulative_returns']=df['Strategy_Returns'].cumsum()
sharpe_ratio = df['Strategy_Returns'].mean()/df['Strategy_Returns'].std()
cumulative_return_cummax = df['cummulative_returns'].cummax()
drawdown = df['cummulative_returns'] - cumulative_return_cummax
print("Sharpe ratio: ", sharpe_ratio)
print("Maximum Drawdown: ", drawdown.min())

Sharpe ratio:  0.014668063918172996
Maximum Drawdown:  -0.03372359296887639


<ipython-input-53-0a10c800fad2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Strategy_Returns'] = df['PnL'].pct_change() * df['Position_Size'].shift(1)
<ipython-input-53-0a10c800fad2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cummulative_returns']=df['Strategy_Returns'].cumsum()


In [54]:
cumulative_return = df['cummulative_returns'].iloc[-1]
print("Cummulative Returns", cumulative_return)

Cummulative Returns 0.03991186009152048
